# Cherry Picking Info: Experiment 2

#  GLAM estimation and out of sample prediction - Individual fit

# 1. Free-exploration Time 

# GLAM model estimation/ 2nd Choice
------------------

## Load data

In [ ]:
# Load data
sufix = 'exp2_2ndChoice_Free'
data = pd.read_csv('data/glam_data/CP2020_GlamData_CP2020_'+sufix+'.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data_len = len(data)
print('Total number of trials: ' + str(data_len))
data.head()

## To run hierarchical model is required to have consecutive participant numbers

In [ ]:
data['subject'] = data.replace(data.subject.unique(), list(range(len(data.subject.unique()))))

## Remove NaNs from dataset

In [ ]:
data = data.dropna()
print ('Percent kept: ' + str(len(data)/data_len))

## Split data in training and test sets

In [ ]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

#test_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_test'+sufix+'.csv'))
#train_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

## Individual GLAM estimation

### No gaze bias ($\gamma$ = 1)

In [ ]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

#if not os.path.exists(str('results/estimates/glam_FF2019_full'+sufix+'.npy')):
glam_full.make_model('individual', gamma_val=1.0, t0_val=0)
glam_full.fit(method='NUTS', tune=1000)
#else:
#    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
#    glam_full.estimates = np.load(str('results/estimates/glam_FF2019_full'+sufix+'.npy'))   

In [ ]:
# Save parameter estimates
np.save(str('Results/Estimates/glam_CP2020_NoBias_indiv_noBias'+sufix+'.npy'), glam_full.estimates)

In [ ]:
pd.DataFrame.from_dict(glam_full.estimates)

## estimate convergence 

In [ ]:
if glam_full.type=='hierarchical':

    rhat_gamma =[] 
    rhat_v = []
    rhat_tau =[] 
    rhat_s =[] 
    
    rhats_params = az.rhat(glam_full.trace, method="folded")

    rhats_params_df = pd.DataFrame()
    rhats_params_df['part'] = train_data.subject.unique()
    rhats_params_df['gamma'] = rhats_params.gamma.values
    rhats_params_df['v'] = rhats_params.v.values
    rhats_params_df['tau'] = rhats_params.tau.values
    rhats_params_df['s'] = rhats_params.s.values
    
    ess_gamma =[] 
    ess_v = []
    ess_tau =[] 
    ess_s =[] 
    
    ess_model = az.ess(glam_full.trace, relative=False)
        
    ess_params_df = pd.DataFrame()
    ess_params_df['part'] = train_data.subject.unique()
    ess_params_df['gamma'] = ess_model.gamma.values
    ess_params_df['v'] = ess_model.v.values
    ess_params_df['tau'] = ess_model.tau.values
    ess_params_df['s'] = ess_model.s.values

In [ ]:
if glam_full.type=='individual':

    rhat_gamma =[] 
    rhat_v = []
    rhat_tau =[] 
    rhat_s =[] 
    
    ess_gamma =[] 
    ess_v = []
    ess_tau =[] 
    ess_s =[] 
    
    part_num =  []
    for i in range(len(glam_full.trace)):
        model_trace = glam_full.trace[i]
        # estimate rhat param
        rhats_params = az.rhat(model_trace, method="folded")
        rhat_gamma.append(rhats_params.gamma.values)
        rhat_v.append(rhats_params.v.values)
        rhat_tau.append(rhats_params.tau.values)
        rhat_s.append(rhats_params.s.values)
        part_num.append(i)
    
        # estimate effective sample size
        ess_model = az.ess(model_trace, relative=False)
        ess_gamma.append(ess_model.gamma.values)
        ess_v.append(ess_model.v.values)
        ess_tau.append(ess_model.tau.values)
        ess_s.append(ess_model.s.values)
        
    rhats_params_df = pd.DataFrame()
    rhats_params_df['gamma'] = rhat_gamma
    rhats_params_df['v'] = rhat_v
    rhats_params_df['tau'] = rhat_tau
    rhats_params_df['s'] = rhat_s
    rhats_params_df['part'] = part_num
    
    ess_params_df = pd.DataFrame()
    ess_params_df['gamma'] = ess_gamma
    ess_params_df['v'] = ess_v
    ess_params_df['tau'] = ess_tau
    ess_params_df['s'] = ess_s
        

In [ ]:
1-rhats_params_df

In [ ]:
ess_params_df

In [ ]:
rhats_params_df.to_csv(str('Results/Convergence/GlamCP2020_indiv_noBias_rhatsParams_'+sufix+'.csv'))
ess_params_df.to_csv(str('Results/Convergence/GlamCP2020_indiv_noBias_essParams_'+sufix+'.csv'))

In [ ]:
full_params = pd.DataFrame(glam_full.estimates)
full_params.to_csv(str('Results/ParamsEstimates/GlamCP2020_params_indiv_noBias_'+sufix+'.csv'))

## estimate WAIC scores

In [ ]:
waic_score = []
for i in range(len(glam_full.trace)):
    waic_score.append(pm.waic(glam_full.trace[i]).waic)

np.save(str('results/waic/glam_CP2020_indiv_noBias_'+ sufix +'.npy'), waic_score)
waic_score

## estimate LOO scores

In [ ]:
loo_score = []
for i in range(len(glam_full.trace)):
    loo_score.append(pm.loo(glam_full.trace[i]).loo)

np.save(str('Results/LOO/glam_CP2020_indiv_noBias_'+ sufix +'.npy'), loo_score)
loo_score

## generate predictions

In [ ]:
print('Predicting test set data using  GLAM...')
glam_full.exchange_data(test_data)

#if not os.path.exists(str('Results/Predictions/GlamPF2020_ind_noBias_'+sufix+'.csv')):
glam_full.predict(n_repeats=50)
glam_full.prediction.to_csv(str('Results/Predictions/GlamCP2020_indiv_noBias_'+sufix+'.csv'), index=False)
#else:
#    print('  Found old individual full GLAM predictions in "results/predictions". Skipping prediction...')
#    glam_full.prediction = pd.read_csv(str('Results/Predictions/GlamPF2020_ind_noBias_'+sufix+'.csv'))

glam_full.prediction.head()

## Plot fit

In [ ]:
#glam.plots_pretty_GLAM.plot_fit(test_data, [glam_full.prediction]);
#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);
glam.plots_pretty_GLAM.plot_fit(test_data, [glam_full.prediction])

plt.show()

# 2. Fixed-exploration Time 

# GLAM model estimation/ 2nd Choice
------------------

## Load data

In [ ]:
# Load data
sufix = 'exp2_2ndChoice_Fixed'
data = pd.read_csv('data/glam_data/CP2020_GlamData_CP2020_'+sufix+'.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data_len = len(data)
print('Total number of trials: ' + str(data_len))
data.head()

## To run hierarchical model is required to have consecutive participant numbers

In [ ]:
data['subject'] = data.replace(data.subject.unique(), list(range(len(data.subject.unique()))))

## Remove NaNs from dataset

In [ ]:
data = data.dropna()
print ('Percent kept: ' + str(len(data)/data_len))

## Split data in training and test sets

In [ ]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

#test_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_test'+sufix+'.csv'))
#train_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

## Individual GLAM estimation

### No gaze bias ($\gamma$ = 1)

In [ ]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

#if not os.path.exists(str('results/estimates/glam_FF2019_full'+sufix+'.npy')):
glam_full.make_model('individual', gamma_val=1.0, t0_val=0)
glam_full.fit(method='NUTS', tune=1000,chains=4,cores=1)
#else:
#    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
#    glam_full.estimates = np.load(str('results/estimates/glam_FF2019_full'+sufix+'.npy'))   

In [ ]:
# Save parameter estimates
np.save(str('Results/Estimates/glam_CP2020_NoBias_indiv_noBias'+sufix+'.npy'), glam_full.estimates)

In [ ]:
pd.DataFrame.from_dict(glam_full.estimates)

## estimate convergence 

In [ ]:
if glam_full.type=='hierarchical':

    rhat_gamma =[] 
    rhat_v = []
    rhat_tau =[] 
    rhat_s =[] 
    
    rhats_params = az.rhat(glam_full.trace, method="folded")

    rhats_params_df = pd.DataFrame()
    rhats_params_df['part'] = train_data.subject.unique()
    rhats_params_df['gamma'] = rhats_params.gamma.values
    rhats_params_df['v'] = rhats_params.v.values
    rhats_params_df['tau'] = rhats_params.tau.values
    rhats_params_df['s'] = rhats_params.s.values
    
    ess_gamma =[] 
    ess_v = []
    ess_tau =[] 
    ess_s =[] 
    
    ess_model = az.ess(glam_full.trace, relative=False)
        
    ess_params_df = pd.DataFrame()
    ess_params_df['part'] = train_data.subject.unique()
    ess_params_df['gamma'] = ess_model.gamma.values
    ess_params_df['v'] = ess_model.v.values
    ess_params_df['tau'] = ess_model.tau.values
    ess_params_df['s'] = ess_model.s.values

In [ ]:
if glam_full.type=='individual':

    rhat_gamma =[] 
    rhat_v = []
    rhat_tau =[] 
    rhat_s =[] 
    
    ess_gamma =[] 
    ess_v = []
    ess_tau =[] 
    ess_s =[] 
    
    part_num =  []
    for i in range(len(glam_full.trace)):
        model_trace = glam_full.trace[i]
        # estimate rhat param
        rhats_params = az.rhat(model_trace, method="folded")
        rhat_gamma.append(rhats_params.gamma.values)
        rhat_v.append(rhats_params.v.values)
        rhat_tau.append(rhats_params.tau.values)
        rhat_s.append(rhats_params.s.values)
        part_num.append(i)
    
        # estimate effective sample size
        ess_model = az.ess(model_trace, relative=False)
        ess_gamma.append(ess_model.gamma.values)
        ess_v.append(ess_model.v.values)
        ess_tau.append(ess_model.tau.values)
        ess_s.append(ess_model.s.values)
        
    rhats_params_df = pd.DataFrame()
    rhats_params_df['gamma'] = rhat_gamma
    rhats_params_df['v'] = rhat_v
    rhats_params_df['tau'] = rhat_tau
    rhats_params_df['s'] = rhat_s
    rhats_params_df['part'] = part_num
    
    ess_params_df = pd.DataFrame()
    ess_params_df['gamma'] = ess_gamma
    ess_params_df['v'] = ess_v
    ess_params_df['tau'] = ess_tau
    ess_params_df['s'] = ess_s
        

In [ ]:
1-rhats_params_df

In [ ]:
ess_params_df

In [ ]:
rhats_params_df.to_csv(str('Results/Convergence/GlamCP2020_indiv_noBias_rhatsParams_'+sufix+'.csv'))
ess_params_df.to_csv(str('Results/Convergence/GlamCP2020_indiv_noBias_essParams_'+sufix+'.csv'))

In [ ]:
full_params = pd.DataFrame(glam_full.estimates)
full_params.to_csv(str('Results/ParamsEstimates/GlamCP2020_params_indiv_noBias_'+sufix+'.csv'))

## estimate WAIC scores

In [ ]:
waic_score = []
for i in range(len(glam_full.trace)):
    waic_score.append(pm.waic(glam_full.trace[i]).waic)

np.save(str('results/waic/glam_CP2020_indiv_noBias_'+ sufix +'.npy'), waic_score)
waic_score

## estimate LOO scores

In [ ]:
loo_score = []
for i in range(len(glam_full.trace)):
    loo_score.append(pm.loo(glam_full.trace[i]).loo)

np.save(str('Results/LOO/glam_CP2020_indiv_noBias_'+ sufix +'.npy'), loo_score)
loo_score

## generate predictions

In [ ]:
print('Predicting test set data using  GLAM...')
glam_full.exchange_data(test_data)

#if not os.path.exists(str('Results/Predictions/GlamPF2020_ind_noBias_'+sufix+'.csv')):
glam_full.predict(n_repeats=50)
glam_full.prediction.to_csv(str('Results/Predictions/GlamCP2020_indiv_noBias_'+sufix+'.csv'), index=False)
#else:
#    print('  Found old individual full GLAM predictions in "results/predictions". Skipping prediction...')
#    glam_full.prediction = pd.read_csv(str('Results/Predictions/GlamPF2020_ind_noBias_'+sufix+'.csv'))

glam_full.prediction.head()

## Plot fit

In [ ]:
#glam.plots_pretty_GLAM.plot_fit(test_data, [glam_full.prediction]);
#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);
glam.plots_pretty_GLAM.plot_fit(test_data, [glam_full.prediction])

plt.show()

# Import toolboxes

In [ ]:
import glam
import pandas as pd
import numpy as np
import os.path
import arviz as az
import seaborn as sns
from IPython.core.pylabtools import figsize
from sklearn import linear_model  # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # 

import matplotlib.pyplot as plt

import pymc3 as pm

np.random.seed(23) # from random.org

def logisticplot_simpl (modlow, data, xaxis='zDV', yaxis='G_choice', ylab='P(Chose Reference Item)', xlab='DV (Z-score)',
                  modlowcol='#AAAAAA', title='empty', xlim = [-5,5]):
    
    sns.set(font_scale=1.5, style='white')
    figsize(5,5)
    
    # defining the sigmoid function
    def model(x):
        y = 1 / (1 + np.exp(-x))
        return y
    
    sub = plt.subplot()


    #run the classifier
    clf = linear_model.LogisticRegression(C=1e5)

    logit_low = {}

    # I think this defines the problem space
    X_test = np.linspace(-10,10,300)

    # fitting the predictive logistic model for the low_confidence trials, for a participant specified by x
    # first I specify the value difference right - left, then I specify the choices, left or right
    clf.fit(data[xaxis][:, np.newaxis],
            data [yaxis])
    logit_low = model(X_test*clf.coef_ + clf.intercept_).ravel()
    print ('Low measure coef',clf.coef_)
    
    #Plotting the predictive lines
    line_low = sub.plot(X_test, logit_low, color=modlowcol, linewidth=5, label=modlow, zorder=5) 
    
    # Set Labels
    sub.set_ylabel(ylab, fontsize=30)
    sub.set_xlabel(xlab, fontsize=30)

    # Set Ticks
    sub.set_xticks((-5,-3,-1,1,3,5))
    sub.set_yticks((0,0.25,0.5,0.75,1))
    sub.tick_params(axis='both', which='major', labelsize=20)

    # Set Limits
    sub.set_ylim(-0.01, 1.01)
    sub.set_xlim(xlim[0], xlim[1])

    # Set Title
    if title == 'empty':
        sub.set_title('')
    else:
        sub.set_title(title)
    
    sub.legend(loc=0, prop={'size':20})
    
    sns.despine()

## [END] 